## Chapter 4 - word2vec 속도 개선

#### 4.1 word2vec 개선 1
---

은닉층 이전의 문제점을 개선해보자


1. 첫 번째 문제점 

	![](../image/image_17.png)

	첫 번째는 입력층의 원핫 표현과 관련된 문제입니다. 단어를 원핫 표현으로 다루기 때문에 어휘 수가 많아지면 원핫 표현의 벡터 크기도 커지는 것이죠. 

	예컨데 어휘가 100만개라면 그 원핫 표현 하나만 해도 원소 수가 100만개인 벡터가 됩니다. 
	
	상당한 메모리를 차지하겠죠. 게다가 이 원핫 벡터와 가중치 행렬 W_in 을 곱해야 하는데, 이것만으로 계산 자원을 상당히 사용하게 됩니다.

	이 문제는 Embedding 계층을 도입하는 것으로 해결이 가능합니다

2. Embedding 계층

	![](../image/image_18.png)

	- 가중치 매개변수로부터 “단어 ID에 해당하는 행(벡터)’을 추출하는 계층

	- 입력한 행을 그냥 출력해주는 계층이다. 




#### 4.2 word2vec 개선 ②
---

은닉층 이후에 문제점을 개선해보자

1. 두 번째 문제점
	
	은닉층의 뉴런과 가중치 행렬(W_out)의 곱

	![](../image/image_19.png)

	첫 번째 문제점은 입력이 원 핫 벡터였기 때문에 그저 뽑아내는 Embedding 계층으로 해결이 가능했지만, 은닉층 이후로는 인코딩된 벡터이기 때문에 Embedding 계층으로 해결이 불가능하다.

2. 세 번째 문제점

	Softmax 계층의 계산

	어휘가 많아지면 softmax의 계산량도 증가합니다. 

	![](../image/image_20.png)


3. 다중 분류에서 이진 분류로

	위 문제점을 해결하려면 네거티브 샘플링 기법이라는 것을 알아야 하는데 이는 다중분류를 이진분류로 근사하는 것이 네거티브 샘플링을 이해하는데 중요한 포인트입니다.

	다중 분류 : 맥락이 'you'와 'goodbye' 일 때, 타깃 단어는 무엇입니까? 

	이진 분류 : 맥락이 'you'와 'goodbye' 일 때, 타깃 단어는 'say' 입니까?

	즉 이진 분류는 "Yes/No"로 답하는 문제를 다룹니다.

	![](../image/image_21.png)

	![](../image/image_22.png)


4. 이진 분류 문제를 신경망으로 풀려면 점수에 시그모이드 함수를 적용해 확률로 변환하고, 손실을 구할 때는 손실 함수로 ‘교차 엔트로피 오차’를 사용합니다.

	다중 분류의 경우, 출력층에서는 (점수를 확률로 변환할 때) ‘소프트 맥스 함수’를 손실 함수로는 ‘교차 엔트로피 오차’를 이용합니다. 
	
	이진 분류의 경우, 줄력층에서는 ‘시그모이드 함수’를, 손실 함수로는 ‘교차 엔트로피 오차’를 이용합니다

	![](../image/image_23.png)

	![](../image/image_24.png)


5. word2vec의 변화

	![](../image/image_25.png)
	
	다중 분류 : 맥락이 'you'와 'goodbye' 일 때, 타깃 단어는 무엇입니까? 

	이진 분류 : 맥락이 'you'와 'goodbye' 일 때, 타깃 단어는 'say' 입니까?

	즉 출력 측의 가중치 W_out에서 단어 'say'에 해당하는 단어 벡터와의 내적을 계산합니다. 그리고 그 출력을 sigmoid with Loss에 입력해 최종 손실을 얻습니다. 


6. 네거티브 샘플링

	지금까지 배운 것으로 주어진 문제를 ‘다중 분류’에서 ‘이진 분류’로 변환할 수 있습니다. 
	
	하지만 안타깝게도 이것만으로는 문제가 다 해결되지 않습니다. 지금까지는 긍정적인 예(정답)에 대해서만 학습했기 때문이죠. 
	
	다시 말해 부정적인 예(오답)를 입력하면 어떤 결과가 나올지 확실하지 않습니다.

	![](../image/image_26.png)

	현재의 신경망에서는 긍정적 예인 ‘say’에 대해서만 학습하게 됩니다. 그러나 부정적 예 (‘say’이외의 단어)에 대해서는 어떠한 지식도 휙득하지 못했죠)

	긍정적 예에 대해서는 sigmoid 계층의 출력을 1에 가깝게 만들고, 부정적 예에 대해서는 sigmoid 계층의 출력을 0에 가깝게 만드는 것입니다.

	![](../image/image_27.png)


	모든 부정적 예를 대상으로 하는 방법은 어휘 수가 늘어나면 감당할 수 없기 때문에(어휘 수 증가에 대처하는 것이 이번 장에서의 목적이었습니다) 

	근사적인 해법으로, 부정적 예를 몇 개 (5개라든지, 10개라든지) 선택합니다.

	즉, 적은 수의 부정적 예를 샘플링해 사용합니다. 이것이 바로 ‘네거티브 샘플링’ 기법이 의미하는 바입니다.

	네거티브 샘플링 기법은 긍정적 예를 타깃으로 한 경우의 손실을 구합니다. 그와 동시에 부정적 예를 몇 개샘플링(선별)하여, 
	
	그 부정적 예에 대해서도 마찬가지로 손실을 구합니다. 그리고 각각의 데이터(긍정적 예와 샘플링된 부정적 예)의 손실을 더한 값을 최종 손실로 합니다.

	부정적 예에 대해서는 sigmoid with Loss 계층에 정답 레이블로 0을 입력합니다

	![](../image/image_28.png)

	![](../image/image_29.png)

	네거티브 샘플링에서는 부정적 예를 가능한 한 많이 다루는 것이 좋습니다만, 계산량 문제 때문에 적은 수(5개나 10개 등)로 한정해야 합니다. 그런데 우연히도 '희소한 단어'만 선택되었다면 어떻게 될까요?

	당연히 결과가 나빠질 것입니다.

	왜냐하면 실전 문제에서도 희소한 단어는 거의 출연하지 않기 떄문이죠, 즉, 드문 단어를 잘 처리하는 일은 중요도가 낮습니다. 그보다는 흔한 단어를 잘 처리하는 편이 좋은 결과로 이러질 것입니다.

	![](../image/image_30.png)

#### 4.3 개선판 word2vec 학습
---

#### 4.4 word2vec 남은 주제
---

